In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.colors import BoundaryNorm

In [2]:
def mass_to_meq(mass_conc, valence, molar_mass, bd):
    return mass_conc * 1000 * valence / molar_mass / 10 / bd

def mass_to_mol(mass_conc, molar_mass, h2o):
    return mass_conc / molar_mass / h2o / 1000.

In [3]:
case = '20240923_conus_ICB20TRCNPRDCTCBC_erw'
# year = 2009
for year in range(1860, 2021, 20):
    hr = xr.open_dataset(
        os.path.join(os.environ['E3SM_ROOT'], 'output', case, 'run', 
                    f'{case}.elm.h1.{year}-01-01-00000.nc')
    )

    layer = 6
    fig, axes = plt.subplots(6, 3, figsize = (20, 20), sharex = True, sharey = True)
    # pH
    ax = axes[0, 0]
    data = hr[f'ceca_col'][:, layer].mean(axis = 0)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('ceca_col (meq 100g-1 dry soil)')

    ax = axes[0, 1]
    data = hr[f'soil_pH'][:, layer].mean(axis = 0)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('soil pH')

    ax = axes[0, 2]
    data = mass_to_meq(hr[f'cec_proton_vr'][:, layer], 1, 1, hr['bd_col'][:, layer]).mean(axis = 0)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('cec_proton_vr (meq 100g-1 dry soil)')

    # cations
    valence_list = [2, 2, 1, 1, 3]
    mass_list = [40, 24, 23, 39, 27]
    for icat in range(1,6):
        ax = axes[icat, 0]
        data = hr[f'cece_col_{icat}'][:, layer].mean(axis = 0)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cece_col_{icat} (meq 100g-1 dry soil)')

        ax = axes[icat, 1]
        data = mass_to_mol(hr[f'cation_vr_{icat}'][:, layer], mass_list[icat-1], 
                        hr['H2OSOI'][:, layer]).mean(axis = 0)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cation_vr_{icat}')

        ax = axes[icat, 2]
        data = mass_to_meq(hr[f'cec_cation_vr_{icat}'][:, layer], valence_list[icat-1], 
                        mass_list[icat-1], hr['bd_col'][:, layer]).mean(axis = 0)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(hr['lon'], hr['lat'], c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cec_cation_vr_{icat} (meq 100g-1 dry soil)')

    plt.savefig(os.path.join(os.environ['PROJDIR'], 'ERW_LDRD', 'results', 'plot_spinupDiags', 
                            f'transient_{year:04d}.png'), dpi = 600., 
                            bbox_inches = 'tight')
    plt.close(fig)
    hr.close()